In [2]:
import numpy as np
import pandas as pd

In [2]:
from energy_consumption.help_functions import handle_outstanding_dp
from energy_consumption.models.seasonal_QR_hourly import get_seasonal_QR_hourly
from energy_consumption.models.seasonal_QR import get_seasonal_QR
from energy_consumption.help_functions.get_energy_data import get_data
from evaluation.help_functions.prepare_data import most_recent_wednesday

energydata = get_data()

100%|██████████| 101/101 [00:59<00:00,  1.71it/s]


In [3]:
energydata

,energy_consumption
date_time,
2021-12-20 00:00:00,49.57150
2021-12-20 01:00:00,47.99850
2021-12-20 02:00:00,47.46700
2021-12-20 03:00:00,47.96675
2021-12-20 04:00:00,49.45700
...,...
2023-11-22 08:00:00,65.75125
2023-11-22 09:00:00,67.40950
2023-11-22 10:00:00,68.26775


In [4]:
# make sure last energy value is on wednesday, 12:00 AM
start_date = most_recent_wednesday(energydata)

# set last timestamp 
energydata_wed12 = energydata.loc[energydata.index <= start_date]
energydata_wed12

,energy_consumption
date_time,
2021-12-20 00:00:00,49.57150
2021-12-20 01:00:00,47.99850
2021-12-20 02:00:00,47.46700
2021-12-20 03:00:00,47.96675
2021-12-20 04:00:00,49.45700
...,...
2023-11-22 08:00:00,65.75125
2023-11-22 09:00:00,67.40950
2023-11-22 10:00:00,68.26775


In [5]:
energy_forecasts_m2 = get_seasonal_QR(energydata)
energy_forecasts_m2

,date_time,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-24 12:00:00,2023-11-24,energy,36 hour,44.807750,53.000477,59.865001,63.504251,68.26325
1,2023-11-24 16:00:00,2023-11-24,energy,40 hour,44.807750,53.000477,59.865001,63.504251,68.26325
2,2023-11-24 20:00:00,2023-11-24,energy,44 hour,40.090751,48.826227,55.199266,58.553500,63.57025
3,2023-11-25 12:00:00,2023-11-24,energy,60 hour,44.807750,53.000477,59.865001,63.504251,68.26325
4,2023-11-25 16:00:00,2023-11-24,energy,64 hour,44.807750,53.000477,59.865001,63.504251,68.26325
5,2023-11-25 20:00:00,2023-11-24,energy,68 hour,40.090751,48.826227,55.199266,58.553500,63.57025


In [3]:
if 'date_time' in energy_forecasts_m2.columns:
    energy_forecasts_m2 = energy_forecasts_m2.drop(columns=['date_time'])
energy_forecasts_m2

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-23,energy,36 hour,44.781250,53.000499,59.865001,63.504255,68.26325
1,2023-11-23,energy,40 hour,40.090751,48.826250,55.199265,58.553504,63.57025
2,2023-11-23,energy,44 hour,35.766249,40.926000,43.832750,47.326500,63.90354
3,2023-11-23,energy,60 hour,44.781250,53.000499,59.865001,63.504255,68.26325
4,2023-11-23,energy,64 hour,40.090751,48.826250,55.199265,58.553504,63.57025
5,2023-11-23,energy,68 hour,35.766249,40.926000,43.832750,47.326500,63.90354


In [2]:
list(range(5))


[0, 1, 2, 3, 4]

In [1]:
from dax.models.quantile_regression import get_quantile_regression_forecasts
df_sub_dax = get_quantile_regression_forecasts()
df_sub_dax

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2023-11-23,2023-11-22,DAX,1 day,-2.678742,-0.474546,0.044688,0.631166,2.278425
2023-11-24,2023-11-22,DAX,2 day,-3.012408,-0.60897,0.1178,0.774698,2.887623
2023-11-27,2023-11-22,DAX,5 day,-3.967045,-0.679798,0.245021,1.067693,3.715867
2023-11-28,2023-11-22,DAX,6 day,-4.445875,-0.771721,0.210363,1.282541,3.462121
2023-11-29,2023-11-22,DAX,7 day,-4.722503,-0.774133,0.323645,1.233462,3.661439


In [3]:
from dax.models.time_series_models.ar_garch import get_garch_11_ar
df_sub_dax = get_garch_11_ar()
df_sub_dax

Iteration:      1,   Func. Count:      6,   Neg. LLF: 20221.713671609155
Iteration:      2,   Func. Count:     15,   Neg. LLF: 425705458476.4596
Iteration:      3,   Func. Count:     23,   Neg. LLF: 2121.3041844436784
Iteration:      4,   Func. Count:     30,   Neg. LLF: 2123.8542260892054
Iteration:      5,   Func. Count:     36,   Neg. LLF: 1685.4298317703115
Iteration:      6,   Func. Count:     42,   Neg. LLF: 1682.7014064906982
Iteration:      7,   Func. Count:     47,   Neg. LLF: 1682.6865026658477
Iteration:      8,   Func. Count:     52,   Neg. LLF: 1682.6858515030933
Iteration:      9,   Func. Count:     57,   Neg. LLF: 1682.685794492621
Iteration:     10,   Func. Count:     62,   Neg. LLF: 1682.6857597398964
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1682.6857597394705
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1682.6857597398964
            Iterations: 11
            Function evaluations: 66
            Gradient evalu

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2023-11-23,2023-11-22,DAX,1 day,-1.595624,-0.508277,0.062255,0.632787,1.720134
2023-11-24,2023-11-22,DAX,2 day,-2.006284,-0.586042,0.159160,0.904363,2.324605
2023-11-27,2023-11-22,DAX,5 day,-2.719025,-0.765757,0.259125,1.284007,3.237276
2023-11-28,2023-11-22,DAX,6 day,-3.413977,-0.967722,0.315831,1.599385,4.045640
2023-11-29,2023-11-22,DAX,7 day,-4.068290,-1.132847,0.407383,1.947612,4.883055


In [5]:
from datetime import datetime
import pandas as pd 

weeks = ['0 week', '1 week', '2 week', '3 week', '4 week']
date_st = (datetime.today().strftime('%Y-%m-%d'))

df_sub_inf = pd.DataFrame({
    "forecast_date": date_st,
    "target": "infections",
    "horizon": weeks,
    "q0.025": np.nan,
    "q0.25": np.nan,
    "q0.5": np.nan,
    "q0.75": np.nan,
    "q0.975": np.nan})

In [6]:
submission_frame = pd.merge(df_sub2_dax, energy_forecasts_m1, how='outer')
submission_frame = pd.merge(submission_frame, df_sub_inf, how='outer')

In [7]:
submission_frame

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-22,DAX,1 day,-1.683155,-0.579231,0.000000,0.579231,1.683155
1,2023-11-22,DAX,2 day,-1.705077,-0.586774,0.000000,0.586774,1.705077
2,2023-11-22,DAX,5 day,-1.726114,-0.594014,0.000000,0.594014,1.726114
3,2023-11-22,DAX,6 day,-1.746314,-0.600966,0.000000,0.600966,1.746314
4,2023-11-22,DAX,7 day,-1.765724,-0.607645,0.000000,0.607645,1.765724
5,2023-11-22,energy,36 hour,51.768500,58.648752,61.206631,63.962502,68.327750
6,2023-11-22,energy,40 hour,47.188250,54.448888,56.939760,59.434001,63.865750
7,2023-11-22,energy,44 hour,38.962261,43.626750,46.693379,50.799008,63.869739
8,2023-11-22,energy,60 hour,51.768500,58.648752,61.206631,63.962502,68.327750
9,2023-11-22,energy,64 hour,47.188250,54.448888,56.939760,59.434001,63.865750


In [8]:
date_string = date_st
date_obj = datetime.strptime(date_string, "%Y-%m-%d")
formatted_date = date_obj.strftime("%Y%m%d")

submission_frame.to_csv(
    str('C:/Users/Maria/Documents/Studium/Pyhton Projekte/PTSFC/submission_files/' +
        formatted_date + '_BrienneVonTarth'),
    index=False, na_rep="NA")